In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

In [4]:
import pandas as pd
store = pd.HDFStore('store.h5')

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [5]:
# Iex finance links:
# 0. https://addisonlynch.github.io/iexfinance/stable/stocks.html#stocks-historical
# 1. HistoricalReader class: https://github.com/addisonlynch/iexfinance/blob/d6d23537029895cc0ea62b25649aeeda67103b9e/iexfinance/stocks/historical.py
# 2. Stock class: https://github.com/addisonlynch/iexfinance/blob/d6d23537029895cc0ea62b25649aeeda67103b9e/iexfinance/stocks/base.py#L8
# 3. _IEXBase: https://github.com/addisonlynch/iexfinance/blob/b23c0646f614c84dc32750aee3bbb3cf10afbe35/iexfinance/base.py#L18

# https://intercom.help/iexcloud/en/articles/4063720-historical-stock-prices-on-iex-cloud
from iexfinance.stocks import get_historical_data

In [6]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
NUM_YEARS_HISTORY = 1
SYMBOL = 'AMZN'
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"

In [7]:
# Assumptions:
# 1. No missing data in existing dataframes in the store; data is complete between (date_min, date_max).
# 2. The requested data is of the same format (URL, params, retrieved columns, etc...).
# Reference for metadata: https://moonbooks.org/Articles/How-to-add-metadata-to-a-data-frame-with-pandas-in-python-/#store-in-a-hdf5-file
def get_historical_data_cached(store, symbol, start_date, end_date, **kwargs):
    def sanitize_date(date):
        return datetime(date.year, date.month, date.day)
    
    start_date = sanitize_date(start_date)
    end_date = sanitize_date(end_date)

#     kwargs['start'] = start_date
#     kwargs['end'] = end_date
    
    print(f"{symbol}: Requesting historical data between {start_date} and {end_date}.")
    
    if symbol not in store:
        print(f"{symbol}: No data is cached.")
        df = get_historical_data(symbol, start_date, end_date, **kwargs)
        metadata = {'min_date': start_date, 'max_date': end_date}
    else:
        df = store[symbol]
        metadata = store.get_storer(symbol).attrs.metadata

        print(f"{symbol}: Data is catched between {metadata['min_date']} and {metadata['max_date']}.")

        if start_date < metadata['min_date']:
            print(f"{symbol}: Getting data between {start_date} and {metadata['min_date']}.")
            df = df.append(get_historical_data(symbol, start_date, metadata['min_date'], **kwargs))
            metadata['min_date'] = start_date
        
        if end_date > metadata['max_date']:
            print(f"{symbol}: Getting data between {metadata['max_date']} and {end_date}.")
            print(f"delta_days = {(datetime.now() - metadata['max_date']).days}")
            df = df.append(get_historical_data(symbol, metadata['max_date'], end_date, **kwargs))
            metadata['max_date'] = end_date            
            
        df = df[~df.index.duplicated(keep='first')]

    store[symbol] = df
    store.get_storer(symbol).attrs.metadata = metadata        

    return df

In [9]:
try:
    os.remove('./store.h5')
except:
    pass

store = pd.HDFStore('store.h5')
SYMBOL = 'AAPL'

end_date = datetime.now() - relativedelta(months=1)
start_date = end_date - relativedelta(years=4)

get_historical_data_cached(store, SYMBOL, start_date, end_date, close_only=True, output_format='pandas', token=IEX_TOKEN)

while end_date < datetime.now():
    # 5 days is the min range https://iexcloud.io/docs/api/#historical-prices
    # iexfinance implementation: https://github.com/addisonlynch/iexfinance/blob/d6d23537029895cc0ea62b25649aeeda67103b9e/iexfinance/stocks/historical.py#L9
    end_date += relativedelta(days=5)
    get_historical_data_cached(store, SYMBOL, start_date, end_date, close_only=True, output_format='pandas', token=IEX_TOKEN)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443


AAPL: Requesting historical data between 2016-10-26 00:00:00 and 2020-10-26 00:00:00.
AAPL: No data is cached.


DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=5y&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG - REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=5y&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG:iexfinance.base:REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=5y&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG - RESPONSE: 200
DEBUG:iexfinance.base:RESPONSE: 200
INFO - MESSAGES USED: 2518
INFO:iexfinance.base:MESSAGES USED: 2518
/Users/olshansky/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->block

AAPL: Requesting historical data between 2016-10-26 00:00:00 and 2020-10-31 00:00:00.
AAPL: Data is catched between 2016-10-26 00:00:00 and 2020-10-26 00:00:00.
AAPL: Getting data between 2020-10-26 00:00:00 and 2020-10-31 00:00:00.
delta_days = 31


DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=3m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG - REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=3m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG:iexfinance.base:REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=3m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG - RESPONSE: 200
DEBUG:iexfinance.base:RESPONSE: 200
INFO - MESSAGES USED: 130
INFO:iexfinance.base:MESSAGES USED: 130
/Users/olshansky/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer-float,key->block0_

...

DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG - REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG:iexfinance.base:REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG - RESPONSE: 200
DEBUG:iexfinance.base:RESPONSE: 200
INFO - MESSAGES USED: 46
INFO:iexfinance.base:MESSAGES USED: 46
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): sandbox.iexapis.com:443


AAPL: Requesting historical data between 2016-10-26 00:00:00 and 2020-11-25 00:00:00.
AAPL: Data is catched between 2016-10-26 00:00:00 and 2020-11-20 00:00:00.
AAPL: Getting data between 2020-11-20 00:00:00 and 2020-11-25 00:00:00.
delta_days = 6


DEBUG:urllib3.connectionpool:https://sandbox.iexapis.com:443 "GET /stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895 HTTP/1.1" 200 None
DEBUG - REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG:iexfinance.base:REQUEST: https://sandbox.iexapis.com/stable/stock/market/batch?symbols=AAPL&types=chart&range=1m&chartByDay=False&chartCloseOnly=True&token=Tpk_57fa15c2c86b4dadbb31e0c1ad1db895
DEBUG - RESPONSE: 200
DEBUG:iexfinance.base:RESPONSE: 200
INFO - MESSAGES USED: 46
INFO:iexfinance.base:MESSAGES USED: 46


AAPL: Requesting historical data between 2016-10-26 00:00:00 and 2020-11-30 00:00:00.
AAPL: Data is catched between 2016-10-26 00:00:00 and 2020-11-25 00:00:00.
AAPL: Getting data between 2020-11-25 00:00:00 and 2020-11-30 00:00:00.
delta_days = 1


ValueError: Start and end dates must be before current date